# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Environment, ScriptRunConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails
from azureml.core.run import Run

import logging
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.core import Workspace, Dataset

subscription_id = 'fa3dfb7e-5583-41a5-b60c-022e3fcc2942'
resource_group = 'mlops-rg-templateml'
workspace_name = 'mlops-aml-ws-templateml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='HealthCareDataset_StrokeData')
data = dataset.to_pandas_dataframe()

In [3]:
experiment_name = 'hyperdrive-stroke-classification-experiment'

experiment = Experiment(workspace, experiment_name)

print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: mlops-aml-ws-templateml
Azure region: eastus
Subscription id: fa3dfb7e-5583-41a5-b60c-022e3fcc2942
Resource group: mlops-rg-templateml


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "F-VM"
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
print("Cluster details: ", cpu_cluster.get_status().serialize())

Found existing cluster, use it.

Running
Cluster details:  {'errors': [], 'creationTime': '2021-02-25T14:58:59.139839+00:00', 'createdBy': {'userObjectId': 'c3dbd685-d45f-431b-b4a5-5c916d4fe4ac', 'userTenantId': '006c1e48-e342-47e9-ab5d-0dd9ff89bd96', 'userName': None}, 'modifiedTime': '2021-02-25T15:01:48.959845+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

#Specify a parameter sampler (Bayesian sampling)
ps = BayesianParameterSampling({'--C': uniform(0.01,1),'--max_iter': choice(100, 150, 200, 250, 300)})

#Create a directory 'training'
if "training" not in os.listdir():
    os.mkdir("./training")

#Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
                compute_target=cpu_cluster,
                entry_script='train.py')

#Create a HyperDriveConfig using the estimator and hyperparameter sampler.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = 'accuracy',
                                   primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs = 20,
                                   max_concurrent_runs = 4,
                                   policy = None,
                                   estimator = est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
#TODO: Submit your experiment
hd_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
from azureml.core.model import Model

#Get the best run.
best_run_hd = hd_run.get_best_run_by_primary_metric()
best_run_metrics_hd = best_run_hd.get_metrics()
print("Best Run Id: ", best_run_hd.id)
print("Accuracy: ", best_run_metrics_hd['accuracy'])

Best Run Id:  HD_00554321-b665-47fb-8310-d00c12e1a881_15
Accuracy:  0.9608610567514677


In [9]:
#TODO: Save the best model
model_hd = best_run_hd.register_model(model_name='hyperdrive_best_model', 
                                model_path='./outputs/model.pkl',
                                model_framework=Model.Framework.SCIKITLEARN, 
                                model_framework_version='0.19.1')
print("Model successfully saved.")

Model successfully saved.
